In [44]:
import numpy as np
import os
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from mpl_toolkits.mplot3d import axes3d
from  PIL import Image
from time import time
from torch import nn
from torchvision import models
from torchvision import transforms
from torchvision import datasets
from torch.utils.data.dataset import Dataset
from torchsummary import summary
import pylab
import numpy 


torch.cuda.is_available()

True

In [2]:
resnet18 = models.resnet18(pretrained=True)

In [3]:
resnet34 = models.resnet34(pretrained=True)

In [6]:
resnet50 = models.resnet50(pretrained=True)

In [4]:
resnet101 = models.resnet101(pretrained=True)

In [5]:
resnet152 = models.resnet152(pretrained=True)

In [45]:
# choose the model.
resnet = resnet18

In [46]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [47]:
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )])

In [48]:
path = r'C:\Users\richa\Desktop\projects\Bristol uni porjects\Univecity projects\Applied Data Science\uob_image_set'

In [64]:
def load_images_from_folder(folder, end, as_tensor = False):

    images = []
    files = folders = 0
    count= 0
    dirnames = [] 

    num_img = []

    for _, dirnames, filenames in os.walk(folder):

 

        if dirnames != []:
            subfolders = dirnames
        current_path = os.path.join(folder, subfolders[count])
        num_img.append(len(os.listdir(current_path)))
        for filename in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, filename))
            if img is not None:
                if as_tensor:
                    img = torch.from_numpy(img * 1.0)
                    images.append(img)
                else: 
                    images.append(img)

 

        count += 1
        
        if count % 10 == 0:
            print('number of folder done =', count, 'total number of images so far =', len(images))
            
        if count  == end:
            return images, num_img

    return images, num_img


def show_image(PIL_img):
    PIL_img.show()

In [65]:
dataset, num_img_list = load_images_from_folder(path, 3)

In [52]:
len(dataset)

13

In [53]:
print(num_img_list), print(sum(num_img_list))

[5, 4, 4]
13


(None, None)

In [54]:
list_input_tensor = []
list_input_batch = []
num_img = sum(num_img_list)
for item in range(num_img):
    input_tensor = preprocess(dataset[item])
    input_batch = input_tensor.unsqueeze(0)
    list_input_tensor.append(input_tensor)
    list_input_batch.append(input_batch)
    

In [55]:
torch.cuda.is_available()

True

In [56]:
for param in resnet.parameters():
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 3)


In [57]:
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    
    with torch.no_grad():
        output = resnet(input_batch)
    print(output)

tensor([[-1.0669, -0.5598, -0.5838]], device='cuda:0')
tensor([[-0.9952, -0.6034, -0.6218]], device='cuda:0')
tensor([[-1.0041, -0.5859, -0.6175]], device='cuda:0')
tensor([[-1.0284, -0.5745, -0.6823]], device='cuda:0')
tensor([[-1.0023, -0.5628, -0.6275]], device='cuda:0')
tensor([[-1.0335, -0.6063, -0.6763]], device='cuda:0')
tensor([[-1.0207, -0.5925, -0.6291]], device='cuda:0')
tensor([[-1.0627, -0.5746, -0.6049]], device='cuda:0')
tensor([[-1.0385, -0.5422, -0.6207]], device='cuda:0')
tensor([[-1.0215, -0.6079, -0.6085]], device='cuda:0')
tensor([[-1.0253, -0.5528, -0.6391]], device='cuda:0')
tensor([[-1.0681, -0.5782, -0.6715]], device='cuda:0')
tensor([[-1.0238, -0.5581, -0.6077]], device='cuda:0')


In [58]:
output_array = []
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    
    with torch.no_grad():
        output = resnet(input_batch)
        cup_tensor = output.cpu()
        
    output_array.append(cup_tensor[0].tolist())
#     output_array.append(np.asarray(cup_tensor[0]))
#     print(np.asarray(cup_tensor[0]))
output_array = np.asarray(output_array)
display(output_array)

array([[-1.06691623, -0.55980444, -0.5838393 ],
       [-0.99520963, -0.60344356, -0.62182832],
       [-1.0041455 , -0.58593249, -0.61753976],
       [-1.02842081, -0.57447708, -0.68234926],
       [-1.00230801, -0.56284624, -0.62745291],
       [-1.03346205, -0.606264  , -0.6762737 ],
       [-1.02071381, -0.59247714, -0.62906897],
       [-1.06269467, -0.57463831, -0.6048733 ],
       [-1.03848684, -0.5421508 , -0.62071443],
       [-1.02154732, -0.60787755, -0.6085149 ],
       [-1.02527428, -0.55281132, -0.63909912],
       [-1.06808639, -0.57817441, -0.67148709],
       [-1.02378571, -0.55810338, -0.60771906]])

In [59]:
import json
import matplotlib.pyplot as plt
import itertools
import random
from scipy.interpolate import *
from mpl_toolkits.mplot3d import axes3d

In [60]:
import pylab
import numpy 
import ipympl

fig = plt.figure()
ax = plt.axes(projection="3d")

img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    ax.scatter(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2],color=random_color,s=20)
    img_counter += n
plt.show()  

ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
fig = plt.figure()
img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], 'o',color=random_color)
    img_counter += n
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
fig = plt.figure()
img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,2], 'o',color=random_color)
    img_counter += n
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
fig = plt.figure()
img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2], 'o',color=random_color)
    img_counter += n
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fig = plt.figure()
plt.plot(output_array[:4,0], output_array[:4,1], 'o',color='r')
plt.plot(output_array[4:,0], output_array[4:,1], 'o',color='b')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig = plt.figure()
plt.plot(output_array[:4,0], output_array[:4,2], 'o',color='r')

plt.plot(output_array[4:,0], output_array[4:,2], 'o',color='b')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
fig = plt.figure()
plt.plot(output_array[:4,1], output_array[:4,2], 'o',color='r')
plt.plot(output_array[4:,1], output_array[4:,2], 'o',color='b')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
import pylab
import numpy 
import ipympl

# %matplotlib widget
# %matplotlib notebook

# x = [item[0] for item in output_array]
# y = [item[1] for item in output_array]
# z = [item[2] for item in output_array]
# # output_array[:,0], output_array[:,1], output_array[:,2]

# output_array_1 = numpy.array(output_array)
# print(output_array_1)

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter(output_array[:4,0], output_array[:4,1], output_array[:4,2],color='r',s=3)
ax.scatter(output_array[4:8,0], output_array[4:8,1], output_array[4:8,2],color='y',s=3)
ax.scatter(output_array[8:,0], output_array[8:,1], output_array[8:,2],color='b',s=3)
ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

# length = data.shape[0]
# width = data.shape[1]
# x, y = np.meshgrid(np.arange(length), np.arange(width))

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1, projection='3d')
# ax.plot_surface(x, y, data)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …